Preparação dos arquivos

In [34]:
from modules.utilities import prepare_data_filesystem

prepare_data_filesystem()

ERROR:root:O diretório data/landing já existe, nada será feito.
ERROR:root:O diretório data/bronze já existe, nada será feito.
ERROR:root:O diretório data/silver já existe, nada será feito.
ERROR:root:O diretório data/gold já existe, nada será feito.


Etapa Landing (Extraindo a lista de IDs)

In [35]:
from modules.etl.extract import landing_step

landing_step(database='nucleotide',term='Chikungunya[title]',retmax=1000,datetype="pdat",reldate=60)

Etapa bronze (Extraindo as informações dos IDs do XML salvo pela etapa Landing)

In [36]:
from modules.etl.transform import bronze_step

bronze_step('nucleotide',rettype='gbc',retmode='xml')

Etapa Silver (Das informações dos IDs retirar o que é interessante e jogar em um formato de arquivo mais adequado para manipulação) -> Em testes ainda

In [45]:
import xmltodict
import pandas as pd
from datetime import datetime

with open("data/bronze/11-04-2024 (1).xml",'r') as xml_file:
    xml_data = xml_file.read()

df = pd.DataFrame(xmltodict.parse(xml_data).get('INSDSet').get('INSDSeq'))

In [46]:
# Convertendo os valores para os tipos de dados mais adequados
df['INSDSeq_locus'] = df['INSDSeq_locus'].astype('string')
df['INSDSeq_length'] = df['INSDSeq_length'].astype('int')
df['INSDSeq_strandedness'] = df['INSDSeq_strandedness'].astype('category')
df['INSDSeq_moltype'] = df['INSDSeq_moltype'].astype('category')
df['INSDSeq_topology'] = df['INSDSeq_topology'].astype('category')
df['INSDSeq_division'] = df['INSDSeq_division'].astype('category')
df['INSDSeq_update-date'] = pd.to_datetime(df['INSDSeq_update-date'],format="%d-%b-%Y").dt.date
df['INSDSeq_create-date'] = pd.to_datetime(df['INSDSeq_create-date'],format="%d-%b-%Y").dt.date
df['INSDSeq_definition'] = df['INSDSeq_definition'].astype('string')
df['INSDSeq_primary-accession'] = df['INSDSeq_primary-accession'].astype('string')
df['INSDSeq_accession-version'] = df['INSDSeq_accession-version'].astype('string')
df['INSDSeq_source'] = df['INSDSeq_source'].astype('string')
df['INSDSeq_organism'] = df['INSDSeq_organism'].astype('string')
df['INSDSeq_taxonomy'] = df['INSDSeq_taxonomy'].astype('string')
df['INSDSeq_comment'] = df['INSDSeq_comment'].astype('string')
df['INSDSeq_sequence'] = df['INSDSeq_sequence'].astype('string')
df['INSDSeq_project'] = df['INSDSeq_project'].astype('string')

In [47]:
# Tratando a coluna other-seqids
df['INSDSeq_other-seqids'] = df['INSDSeq_other-seqids'].apply(lambda x: x.get('INSDSeqid') if x is not None else None) # Desmembrando a coluna other-seqids
df = df.explode(column='INSDSeq_other-seqids') # Explodindo cada uma das listas

In [48]:
# Tratando a coluna references
df['INSDSeq_references'] = df['INSDSeq_references'].apply(lambda x: x.get('INSDReference') if isinstance(x, dict) and 'INSDReference' in x.keys() else None)
df['INSDSeq_references'] = df['INSDSeq_references'].apply(lambda x: [x] if not isinstance(x, list) else x)

df = df.explode(column='INSDSeq_references') # Explodindo o a coluna references

# Desmembrando a coluna references
reference_key = "INSDReference_reference"
df['reference_id'] =  df['INSDSeq_references'].apply(lambda dictionary: int(dictionary.get(reference_key)) if reference_key in dictionary.keys() else None)
position_key = "INSDReference_position"
df['reference_position'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(position_key) if position_key in dictionary.keys() else None)
authors_key = "INSDReference_authors"
df['reference_authors'] = df['INSDSeq_references'].apply(lambda dictionary: " | ".join(dictionary.get(authors_key).get("INSDAuthor")) if authors_key in dictionary.keys() else None)
title_key = "INSDReference_title"
df['reference_title'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(title_key) if title_key in dictionary.keys() else None)
journal_key = "INSDReference_journal"
df['reference_journal'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(journal_key) if journal_key in dictionary.keys() else None)
xref_key = "INSDReference_xref"
df['reference_xref_dbname'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(xref_key).get("INSDXref").get("INSDXref_dbname") if xref_key in dictionary.keys() else None)
df['reference_xref_id'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(xref_key).get("INSDXref").get("INSDXref_id") if xref_key in dictionary.keys() else None)
pubmed_key = "INSDReference_pubmed"
df['reference_pubmed'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(pubmed_key) if pubmed_key in dictionary.keys() else None)
remark_key = "INSDReference_remark"
df['reference_remark'] = df['INSDSeq_references'].apply(lambda dictionary: dictionary.get(remark_key) if remark_key in dictionary.keys() else None)
df.drop(columns='INSDSeq_references',inplace=True) # Retirando a coluna original


In [49]:
# Desmembrando a coluna feature-table
df['INSDSeq_feature-table'] = df['INSDSeq_feature-table'].apply(lambda x: x.get("INSDFeature") if isinstance(x, dict) and "INSDFeature" in x.keys() else None)
df['INSDSeq_feature-table'] = df['INSDSeq_feature-table'].apply(lambda x: [x] if not isinstance(x, list) else x)

df = df.explode(column="INSDSeq_feature-table")


,INSDSeq_locus,INSDSeq_length,INSDSeq_strandedness,INSDSeq_moltype,INSDSeq_topology,INSDSeq_division,INSDSeq_update-date,INSDSeq_create-date,INSDSeq_definition,INSDSeq_primary-accession,...,INSDSeq_xrefs,reference_id,reference_position,reference_authors,reference_title,reference_journal,reference_xref_dbname,reference_xref_id,reference_pubmed,reference_remark
0,OP168370,11768,single,RNA,linear,VRL,2024-04-06,2024-04-06,"Chikungunya virus isolate Mdy_95_2019, complet...",OP168370,...,NaN,1,1..11768,"Ngwe Tun,M.M. | Kyaw,A.K. | Nabeshima,T. | Dum...",Coinfection and circulation of chikungunya vir...,"Microbes Infect 25 (6), 105129 (2023)",doi,10.1016/j.micinf.2023.105129,37030472,None
0,OP168370,11768,single,RNA,linear,VRL,2024-04-06,2024-04-06,"Chikungunya virus isolate Mdy_95_2019, complet...",OP168370,...,NaN,1,1..11768,"Ngwe Tun,M.M. | Kyaw,A.K. | Nabeshima,T. | Dum...",Coinfection and circulation of chikungunya vir...,"Microbes Infect 25 (6), 105129 (2023)",doi,10.1016/j.micinf.2023.105129,37030472,None
0,OP168370,11768,single,RNA,linear,VRL,2024-04-06,2024-04-06,"Chikungunya virus isolate Mdy_95_2019, complet...",OP168370,...,NaN,1,1..11768,"Ngwe Tun,M.M. | Kyaw,A.K. | Nabeshima,T. | Dum...",Coinfection and circulation of chikungunya vir...,"Microbes Infect 25 (6), 105129 (2023)",doi,10.1016/j.micinf.2023.105129,37030472,None
0,OP168370,11768,single,RNA,linear,VRL,2024-04-06,2024-04-06,"Chikungunya virus isolate Mdy_95_2019, complet...",OP168370,...,NaN,2,1..11768,"Ngwe Tun,M.M. | Kyaw,A.K. | Nabeshima,T. | Tha...",Direct Submission,Submitted (05-AUG-2022) Virology Research Divi...,None,None,None,None
0,OP168370,11768,single,RNA,linear,VRL,2024-04-06,2024-04-06,"Chikungunya virus isolate Mdy_95_2019, complet...",OP168370,...,NaN,2,1..11768,"Ngwe Tun,M.M. | Kyaw,A.K. | Nabeshima,T. | Tha...",Direct Submission,Submitted (05-AUG-2022) Virology Research Divi...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,PP236105,11790,single,RNA,linear,VRL,2024-02-11,2024-02-11,Chikungunya virus strain IMR04-21 non-structur...,PP236105,...,NaN,1,1..11790,"Kalyanasundram,J. | Kamel,K.A. | Mohd Zawawi,Z...",Direct Submission,"Submitted (30-JAN-2024) Unit Virology, Infecti...",None,None,None,None
192,PP236105,11790,single,RNA,linear,VRL,2024-02-11,2024-02-11,Chikungunya virus strain IMR04-21 non-structur...,PP236105,...,NaN,1,1..11790,"Kalyanasundram,J. | Kamel,K.A. | Mohd Zawawi,Z...",Direct Submission,"Submitted (30-JAN-2024) Unit Virology, Infecti...",None,None,None,None
192,PP236105,11790,single,RNA,linear,VRL,2024-02-11,2024-02-11,Chikungunya virus strain IMR04-21 non-structur...,PP236105,...,NaN,1,1..11790,"Kalyanasundram,J. | Kamel,K.A. | Mohd Zawawi,Z...",Direct Submission,"Submitted (30-JAN-2024) Unit Virology, Infecti...",None,None,None,None
192,PP236105,11790,single,RNA,linear,VRL,2024-02-11,2024-02-11,Chikungunya virus strain IMR04-21 non-structur...,PP236105,...,NaN,1,1..11790,"Kalyanasundram,J. | Kamel,K.A. | Mohd Zawawi,Z...",Direct Submission,"Submitted (30-JAN-2024) Unit Virology, Infecti...",None,None,None,None


In [51]:
#df["feature_key"] = df['INSDSeq_feature-table'].apply(lambda x:x.get("INSDFeature_key"))

for i in df['INSDSeq_feature-table']:
    print(type(i))

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'di